# Setting up the environment

## clone the project

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p codes
%cd /content/drive/MyDrive/codes
# !git clone https://github.com/navidmdn/logic_based_qa.git
%cd logic_based_qa

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
/content/drive/MyDrive/codes
/content/drive/MyDrive/codes/logic_based_qa


## install requirements and SWI prolog

In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/yuce/pyswip (to revision master) to /tmp/pip-install-j8wiud06/pyswip_ab6c177004f04ed6bcd49c9c028e07ae
  Running command git clone --filter=blob:none --quiet https://github.com/yuce/pyswip /tmp/pip-install-j8wiud06/pyswip_ab6c177004f04ed6bcd49c9c028e07ae
  Resolved https://github.com/yuce/pyswip to commit 59016e0841f56177d1b18ec08fd9b67792bd0a97
  Preparing metadata (setup.py) ... done


In [3]:
!sudo apt-add-repository ppa:swi-prolog/stable
!sudo apt-get update
!sudo apt-get install swi-prolog

 Comprehensive Prolog implementation with extensive libraries and development tools.   Primarily targetted at teaching, RDF processing and web-related tasks, such as creating web services or analysing web content.

Official PPAs for SWI-Prolog. See https://www.swi-prolog.org for further information.
 More info: https://launchpad.net/~swi-prolog/+archive/ubuntu/stable
Press [ENTER] to continue or Ctrl-c to cancel adding it.

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal InRel

# Training language to predicate translator

## build and save translation dataset

In [5]:
!PYTHONPATH=. python nl2log/data_loader.py --data_path=./data --dataset=metaqa --sample_size 1000

100% 134741/134741 [00:00<00:00, 253711.42it/s]
loaded 269482 triplets with 43234 entities and 18 relations
100% 269482/269482 [00:00<00:00, 1141107.13it/s]
train set size:329282, test set size: 39093, dev set size: 39138


## training the translator

In [10]:
!bash translation_trainer.sh 1000

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
Moving 0 files to the new cache system
0it [00:00, ?it/s]
INFO:__main__:Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=2000,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None

## evaluate the translator model 

In [ ]:
!PYTHONPATH=. python nl2log/evaluation.py --model_cp=models/t5-small-1000

# Question Answering Module

## generate predicates from test questions

In [14]:
!PYTHONPATH=. python qa/evaluation.py --model_path="./models/t5-small/checkpoint-500" --generate_predicates

Generating predicates
100% 134741/134741 [00:00<00:00, 247241.43it/s]
loaded 269482 triplets with 43234 entities and 18 relations
100% 269482/269482 [00:00<00:00, 1267720.47it/s]
100% 269482/269482 [00:05<00:00, 46406.15it/s]
100% 39/39 [00:23<00:00,  1.63it/s]
100% 59/59 [00:48<00:00,  1.20it/s]
100% 56/56 [01:03<00:00,  1.13s/it]


## evaluating final model

In [ ]:
!PYTHONPATH=. python qa/evaluation.py --model_path="./models/t5-small/checkpoint-500"

## Manually query model

In [36]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:

from qa.question_answering import QuestionAnswering
from qa.data_loader import MetaQADataLoader

data_loader = MetaQADataLoader('./data')
qa = QuestionAnswering('./models/t5-small/checkpoint-5000', data_loader)

100%|██████████| 134741/134741 [00:00<00:00, 174335.64it/s]


loaded 269482 triplets with 43234 entities and 18 relations


100%|██████████| 269482/269482 [00:08<00:00, 32746.59it/s]


In [7]:
!head -n 5 data/3hop/qa_test.txt

the films that share directors with the film [Catch Me If You Can] were in which languages	German|Polish|Mende|Japanese
who starred movies for the director of [Written on the Wind]	Sandra Dee|Charles Coburn|Cornel Wilde|John Gavin|Warren William|Susan Kohner|Joan Bennett|Fred MacMurray|Barbara Stanwyck|Don Ameche|Jane Wyman|Rochelle Hudson|Boris Karloff|Patricia Knight|Robert Cummings|Rock Hudson|Lucille Ball|Claudette Colbert|Lana Turner|George Sanders
the films that share actors with the film [Creepshow] were in which languages	Polish|English
what were the release years of the films that share writers with the film [Grown Ups 2]	1995|1996|1999|1998|1989|2002|2000|2008|2011|2010
who is listed as director of the films starred by [The Inner Circle] actors	John Landis|Atom Egoyan|Gary Trousdale|Steven Spielberg|Louis Leterrier|Alan Alda|Nora Ephron|Wolfgang Petersen|Allen Coulter|Richard Benjamin|Orson Welles|Neil Jordan|William Dieterle|Stephen Frears|Peter Yates|Markus Schleinzer|Wayne

In [8]:
qa.answer_question(
    "the films that share actors with the film [Creepshow] were in which languages"
)

the films that share actors with the film [Creepshow] were in which languages


['English', 'Polish']

In [9]:
qa.answer_question(
    "what are the languages of the films that have the same actors as [Creepshow]?"
)

what are the languages of the films that have the same actors as [Creepshow]?


['English', 'Polish']